
# **Infraestrutura Escolar (INEP 2024) — Notebook em Python**
Este notebook converte os códigos em **R** enviados por você para **Python**, pronto para rodar no Jupyter.  
Ele cobre:
1) **Carga e limpeza** dos microdados (`microdados_ed_basica_2024.csv`)  
2) **Gráfico de barras**: infraestrutura básica por **região** (água, energia, esgoto)  
3) **Mapas (choropleth)** por **UF** e **rede (Pública x Privada)** usando `geobr` + `geopandas`  
4) **Barras comparativas**: **biblioteca** por região (Pública x Privada)  
5) **Barras comparativas**: **refeitório** por região (Pública x Privada)  
6) **Barras comparativas**: **banheiro** por região (Urbana x Rural)

> **Observações**
> - A estrutura do CSV deve ser a mesma do R (delimitador `;`, encoding `Latin1`).  
> - Para os mapas, utilizo o pacote **`geobr` (Python)**, que baixa os shapefiles oficiais do IBGE. Na primeira execução, ele fará o download dos arquivos.  
> - As paletas seguem um **estilo acadêmico azul**, mantendo legibilidade (similar à sua versão R).


In [ ]:

# ============================================================
# 0) Instalação de pacotes (execute uma única vez, se precisar)
# ============================================================
# Se algum pacote não estiver instalado, descomente as linhas abaixo e rode a célula.
# %pip install pandas numpy matplotlib seaborn plotnine geopandas pyproj shapely geobr mapclassify fiona

import sys, importlib

def ensure(pkg):
    try:
        importlib.import_module(pkg)
        print(f"OK - {pkg}")
    except ImportError:
        print(f"Falta instalar: {pkg}. Rode a célula de instalação acima.")

for p in ["pandas","numpy","matplotlib","geopandas","geobr","mapclassify","plotnine"]:
    ensure(p)


In [ ]:

# ======================================
# 1) Imports e configurações de estilo
# ======================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Opcional: plotnine (estilo ggplot em Python)
from plotnine import (
    ggplot, aes, geom_col, position_dodge, labs, theme_minimal, theme,
    element_text, scale_fill_manual, scale_y_continuous, guides, guide_legend
)

# Geoespacial
import geopandas as gpd
from geobr import read_state

# Formatação
plt.rcParams["figure.dpi"] = 120
plt.rcParams["font.size"] = 11
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False


## 2) Carregar microdados (mesmo arquivo do R)

In [ ]:

# Ajuste o caminho se necessário — o arquivo deve estar na mesma pasta do notebook
ARQ = "microdados_ed_basica_2024.csv"

# O arquivo em R foi lido com delim=';' e encoding Latin1
dados = pd.read_csv(
    ARQ, 
    sep=';', 
    encoding='latin1', 
    dtype=str  # lê tudo como string inicialmente para evitar problemas de tipo
)

# Converte colunas numéricas de interesse para 0/1 (quando aplicável)
# Se já vierem como 0/1 em string, converte abaixo; se vierem como 'S'/'N', ajustar conforme a necessidade.
num_cols = [
    "IN_AGUA_POTAVEL","IN_ENERGIA_REDE_PUBLICA","IN_ESGOTO_REDE_PUBLICA",
    "IN_BANHEIRO","IN_BIBLIOTECA_SALA_LEITURA","IN_REFEITORIO","IN_QUADRA_ESPORTES"
]

for c in num_cols:
    if c in dados.columns:
        dados[c] = pd.to_numeric(dados[c], errors="coerce")

# Seleciona as variáveis de interesse (mesmo do R)
cols_keep = [
    "SG_UF","CO_UF","NO_REGIAO","TP_LOCALIZACAO","TP_DEPENDENCIA",
    "IN_AGUA_POTAVEL","IN_ENERGIA_REDE_PUBLICA","IN_ESGOTO_REDE_PUBLICA",
    "IN_BANHEIRO","IN_BIBLIOTECA_SALA_LEITURA","IN_REFEITORIO","IN_QUADRA_ESPORTES"
]

base_filtrada = dados[cols_keep].dropna()
base_filtrada.head()


## 3) Infraestrutura básica por região (Água, Energia, Esgoto)

In [ ]:

# Calcula percentuais por NO_REGIAO
def pct_true(series):
    # Considera "1" como verdadeiro (igual ao R)
    return (series == 1).mean() * 100

infra_regiao = (
    base_filtrada
      .groupby("NO_REGIAO", dropna=False)
      .agg(
          **{
              "Água potável": ("IN_AGUA_POTAVEL", pct_true),
              "Energia elétrica": ("IN_ENERGIA_REDE_PUBLICA", pct_true),
              "Esgotamento sanitário": ("IN_ESGOTO_REDE_PUBLICA", pct_true),
          }
      )
      .reset_index()
)

infra_long = infra_regiao.melt(id_vars="NO_REGIAO", var_name="Infraestrutura", value_name="Percentual")

infra_regiao, infra_long.head()


In [ ]:

# Gráfico (plotnine) com paleta azul acadêmica
palette = {
    "Água potável": "#1E88E5",
    "Energia elétrica": "#0D47A1",
    "Esgotamento sanitário": "#90A4AE"
}

p = (
    ggplot(infra_long, aes(x="NO_REGIAO", y="Percentual", fill="Infraestrutura"))
    + geom_col(position=position_dodge(width=0.8), width=0.7, alpha=0.9)
    + scale_fill_manual(values=palette)
    + scale_y_continuous(labels=lambda l: [f"{v:.0f}%" for v in l])
    + labs(
        title="Infraestrutura Básica das Escolas Brasileiras – 2024",
        x="Região do Brasil",
        y="% de Escolas com o Serviço",
        fill="Indicador",
        caption="Fonte: INEP – Censo Escolar 2024",
    )
    + theme_minimal(base_size=13)
    + theme(
        figure_size=(9,4.5),
        plot_title=element_text(weight="bold", size=15),
        legend_title=element_text(weight="bold"),
    )
)
p


## 4) Mapas — Pública x Privada (média de água, energia, esgoto)

In [ ]:

# Mapeia a variável TP_DEPENDENCIA para rótulos
# Em R: 4 = Privada; 1/2/3 = Pública
def map_rede(v):
    try:
        v = int(v)
    except:
        pass
    if v == 4 or v == "4":
        return "Privada"
    if v in [1,2,3,"1","2","3"]:
        return "Pública"
    return str(v)

bf = base_filtrada.copy()
bf["REDE"] = bf["TP_DEPENDENCIA"].apply(map_rede)

def pct(series):
    return (series == 1).mean() * 100

infra_mapa = (
    bf.groupby(["SG_UF","REDE"], dropna=False)
      .agg(
          agua=("IN_AGUA_POTAVEL", pct),
          energia=("IN_ENERGIA_REDE_PUBLICA", pct),
          esgoto=("IN_ESGOTO_REDE_PUBLICA", pct),
      )
      .reset_index()
)

infra_mapa["infra_media"] = (infra_mapa["agua"] + infra_mapa["energia"] + infra_mapa["esgoto"]) / 3.0

infra_mapa.head()


In [ ]:

# Lê shapefile de estados via geobr (IBGE)
estados = read_state()  # retorna GeoDataFrame com 'abbrev_state', 'name_state', 'geometry'

# Junta por SG_UF ~ abbrev_state
mapa_infra = estados[["abbrev_state","name_state","geometry"]].merge(
    infra_mapa, left_on="abbrev_state", right_on="SG_UF", how="left"
)

# Ajusta ordenação de REDE nos facet plots (Privada, Pública)
mapa_infra["REDE"] = pd.Categorical(mapa_infra["REDE"], categories=["Privada","Pública"], ordered=True)

mapa_infra.head()


In [ ]:

# Plot facet em 2 colunas: Privada x Pública
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)

# Paleta gradiente (equivalente à usada no R)
# Como matplotlib não aceita gradiente direto por default, usamos 'Blues' e definimos limites 50-100
import matplotlib as mpl
vmin, vmax = 50, 100
cmap = mpl.cm.Blues

for i, rede in enumerate(["Privada","Pública"]):
    ax = axes[i]
    gdf = mapa_infra[mapa_infra["REDE"] == rede]
    gdf.plot(
        column="infra_media",
        ax=ax,
        cmap=cmap,
        vmin=vmin, vmax=vmax,
        edgecolor="white", linewidth=0.3,
        missing_kwds={"color":"lightgrey", "edgecolor":"white", "hatch":"///", "label":"Sem dados"}
    )
    ax.set_axis_off()
    ax.set_title(f"{rede}", fontweight="bold")

# Barra de cor única para ambos
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm, ax=axes, fraction=0.046, pad=0.02)
cbar.set_label("Infraestrutura (%)")

fig.suptitle("Infraestrutura Básica — Pública x Privada (UF)\nMédia de Água, Energia e Esgoto", fontsize=14, fontweight="bold")
plt.show()


## 5) Biblioteca/Sala de Leitura — por região e rede (Pública x Privada)

In [ ]:

tb = base_filtrada.copy()
# REDE
tb["rede"] = tb["TP_DEPENDENCIA"].apply(lambda v: "Privada" if str(v)=="4" else "Pública")

# REGIÃO
tb["regiao"] = tb["NO_REGIAO"]

tab_bib = (
    tb.groupby(["regiao","rede"], dropna=False)
      .agg(pct_biblioteca=("IN_BIBLIOTECA_SALA_LEITURA", lambda s: (s==1).mean()*100))
      .reset_index()
)

# Ordena regiões como no R
ordem = ["Norte","Nordeste","Centro-Oeste","Sul","Sudeste"]
tab_bib["regiao"] = pd.Categorical(tab_bib["regiao"], categories=ordem, ordered=True)
tab_bib = tab_bib.sort_values(["regiao","rede"])

tab_bib.head()


In [ ]:

# Barras com rótulo
fig, ax = plt.subplots(figsize=(9,4))

regioes = tab_bib["regiao"].cat.categories.tolist()
larg = 0.35

pub = tab_bib[tab_bib["rede"]=="Pública"].set_index("regiao")["pct_biblioteca"].reindex(regioes)
priv = tab_bib[tab_bib["rede"]=="Privada"].set_index("regiao")["pct_biblioteca"].reindex(regioes)

x = np.arange(len(regioes))
ax.bar(x - larg/2, pub, width=larg, alpha=0.9, label="Pública")
ax.bar(x + larg/2, priv, width=larg, alpha=0.9, label="Privada")

# Rótulos
for i, (vp, vq) in enumerate(zip(pub, priv)):
    if pd.notna(vp):
        ax.text(i - larg/2, vp + 1, f"{vp:.1f}%", ha="center", va="bottom", fontsize=9)
    if pd.notna(vq):
        ax.text(i + larg/2, vq + 1, f"{vq:.1f}%", ha="center", va="bottom", fontsize=9)

ax.set_xticks(x, regioes)
ax.set_ylim(0, 100)
ax.set_ylabel("% de escolas com biblioteca/sala de leitura")
ax.set_title("Escolas com Biblioteca ou Sala de Leitura — Pública x Privada")
ax.legend(title=None)

plt.show()


## 6) Refeitório — por região e rede (Pública x Privada)

In [ ]:

tab_ref = (
    tb.groupby(["regiao","rede"], dropna=False)
      .agg(pct_ref=("IN_REFEITORIO", lambda s: (s==1).mean()*100))
      .reset_index()
)

tab_ref["regiao"] = pd.Categorical(tab_ref["regiao"], categories=ordem, ordered=True)
tab_ref = tab_ref.sort_values(["regiao","rede"])

fig, ax = plt.subplots(figsize=(9,4))

pub = tab_ref[tab_ref["rede"]=="Pública"].set_index("regiao")["pct_ref"].reindex(regioes)
priv = tab_ref[tab_ref["rede"]=="Privada"].set_index("regiao")["pct_ref"].reindex(regioes)

x = np.arange(len(regioes))
larg = 0.35

ax.bar(x - larg/2, pub, width=larg, alpha=0.9, label="Pública")
ax.bar(x + larg/2, priv, width=larg, alpha=0.9, label="Privada")

for i, (vp, vq) in enumerate(zip(pub, priv)):
    if pd.notna(vp):
        ax.text(i - larg/2, vp + 1, f"{vp:.1f}%", ha="center", va="bottom", fontsize=9)
    if pd.notna(vq):
        ax.text(i + larg/2, vq + 1, f"{vq:.1f}%", ha="center", va="bottom", fontsize=9)

ax.set_xticks(x, regioes)
ax.set_ylim(0, 100)
ax.set_ylabel("% de escolas com refeitório")
ax.set_title("Escolas com Refeitório — Pública x Privada")
ax.legend(title=None)

plt.show()


## 7) Banheiro — por região e localização (Urbana x Rural)

In [ ]:

tb2 = base_filtrada.copy()
# Local: 1 = Urbana, 2 = Rural (igual ao R)
def map_local(v):
    try:
        v = int(v)
    except:
        pass
    if v == 1 or v == "1":
        return "Urbana"
    if v == 2 or v == "2":
        return "Rural"
    return str(v)

tb2["regiao"] = tb2["NO_REGIAO"]
tb2["local"]  = tb2["TP_LOCALIZACAO"].apply(map_local)

tab_ban = (
    tb2.groupby(["regiao","local"], dropna=False)
       .agg(pct_ban=("IN_BANHEIRO", lambda s: (s==1).mean()*100))
       .reset_index()
)

tab_ban["regiao"] = pd.Categorical(tab_ban["regiao"], categories=ordem, ordered=True)
tab_ban = tab_ban.sort_values(["regiao","local"])

fig, ax = plt.subplots(figsize=(9,4))

urb = tab_ban[tab_ban["local"]=="Urbana"].set_index("regiao")["pct_ban"].reindex(regioes)
rur = tab_ban[tab_ban["local"]=="Rural"].set_index("regiao")["pct_ban"].reindex(regioes)

x = np.arange(len(regioes))
larg = 0.35

ax.bar(x - larg/2, urb, width=larg, alpha=0.9, label="Urbana", color="#1E88E5")
ax.bar(x + larg/2, rur, width=larg, alpha=0.9, label="Rural", color="#90A4AE")

for i, (vu, vr) in enumerate(zip(urb, rur)):
    if pd.notna(vu):
        ax.text(i - larg/2, vu + 1, f"{vu:.1f}%", ha="center", va="bottom", fontsize=9)
    if pd.notna(vr):
        ax.text(i + larg/2, vr + 1, f"{vr:.1f}%", ha="center", va="bottom", fontsize=9)

ax.set_xticks(x, regioes)
ax.set_ylim(0, 100)
ax.set_ylabel("% de escolas com banheiro")
ax.set_title("Acesso a Banheiro — Urbana x Rural (Censo Escolar 2024)")
ax.legend(title=None)

plt.show()
